Notebook utilizado para apresentar uma breve introdução ao processamento de sinais.
Criado para a disciplina Geofísica I da Universidade Federal de Uberlândia.
O uso e a reprodução são livres para fins educacionais, pede-se apenas a citação da fonte.

[![License: GPL v3](https://img.shields.io/badge/License-GPLv3-blue.svg)](https://www.gnu.org/licenses/gpl-3.0)

Prof. Dr. Emerson Rodrigo Almeida<br>
Universidade Federal de Uberlândia<br>
Instituto de Geografia - Curso de Geologia<br>
emerson.almeida@ufu.br<br>

<p>&nbsp;</p>
Última modificação: 25/02/2021

# Instruções iniciais

Neste notebook você irá aprender como funciona a remoção de componentes de frequências indesejadas em sinais através do processo de filtragem. O conceito de filtragem é comum a TODOS os métodos geofísicos, portanto é fundamental que você compreenda o que está sendo abordado aqui.

Não se preocupe em compreender o código em si, não é obrigatório que você conheça a programação para obter e interpretar os resultados da demonstração. Mesmo assim, sinta-se livre caso queira perguntar qualquer coisa para aprender mais sobre o código e sua elaboração.

<p>&nbsp;</p>

### Passo 01 - Preparação

Primeiramente vamos preparar o notebook com os módulos e funções que precisaremos no decorrer da demonstração.

Você pode executar o código na célula clicando sobre ela e em seguida clicando no botão **RUN** na barra do Jupyter Notebook ou pressionando as teclas **SHIFT+ENTER** no seu teclado.

<p>&nbsp;</p>

**Importante!** Nem todas as células irão retornar algum tipo de mensagem ou figura após a execução. Isto é normal e não significa que a execução tenha falhado. Perceba o símbolo **In [   ]:** próximo ao canto superior esquerdo de cada célula. Ao iniciar a execução de uma delas você verá um asterisco ($*$) aparecer dentro dos colchetes. Você saberá que a execução ocorreu sem problemas quando o $*$ dentro dos colchetes for substituído por um número e nenhuma mensagem de erro aparecer abaixo da célula executada. 

In [ ]:
import modulos.filtragem as mf
import numpy as np

# Passo 02 - Sinal sintético

Primeiramente vamos criar um sinal sintético para podermos trabalhar. Este sinal será composto por uma sobreposição de cinco componentes, cada uma sendo uma função senóide com suas próprias características de amplitude ($A_1, A_2, ..., A_5$), dada em unidades de amplitude (ua) e de frequência ($f_1, f_2, ..., f_5$), dada em Hertz (Hz). Isto nos permitirá saber o que exatamente estaremos removendo do sinal e o que estamos deixando intacto. Assim, teremos:

$$ s_1(t) = A_1sen(2 \pi f_1 t)$$
$$ s_2(t) = A_2sen(2 \pi f_2 t)$$
$$ s_3(t) = A_3sen(2 \pi f_3 t)$$
$$ s_4(t) = A_4sen(2 \pi f_4 t)$$
$$ s_5(t) = A_5sen(2 \pi f_5 t)$$

$$ y(t) = s_1(t) + s_2(t) + s_3(t) + s_4(t) + s_5(t)$$

em que $y(t)$ é o nosso sinal sintético. As componentes de sinal que iremos utilizar têm amplitudes $A_1$ = 1.0 ua, $A_2$ = 0.5 ua, $A_3$ = 1.5 ua, $A_4$ = 2.75 ua e $A_5$ = 0.8 ua e frequências $f_1$ = 1.0 Hz, $f_2$ = 3.0 Hz, $f_3$ = 15.0 Hz, $f_4$ = 0.50 Hz e $f_5$ = 7.0 Hz. 

Execute a célula abaixo para estabelecer as configurações dos parâmetros do sinal sinal sintético e visualizá-lo.

In [ ]:
a1 = 1.00      # amplitude da componente s_1(t) em unidades de amplitude
f1 = 1.00      # frequência da componente s_1(t) em Hertz

a2 = 0.50      # amplitude da componente s_2(t) em unidades de amplitude
f2 = 3.00      # frequência da componente s_2(t) em Hertz

a3 = 1.50      # amplitude da componente s_3(t) em unidades de amplitude
f3 = 15.0      # frequência da componente s_3(t) em Hertz

a4 = 2.75      # amplitude da componente s_4(t) em unidades de amplitude
f4 = 0.50      # frequência da componente s_4(t) em Hertz

a5 = 0.80      # amplitude da componente s_5(t) em unidades de amplitude
f5 = 7.00      # frequência da componente s_5(t) em Hertz

janela_tempo = 20.0              # janela temporal em segundos


sinal, tempo = mf.gera_sinal([a1, a2, a3, a4, a5], [f1, f2, f3, f4, f5], janela_tempo)
mf.plota_sinal(tempo, sinal, n_figura='1')

# Passo 03 - Mudança de domínio

Perceba que no sinal apresentado na Figura 1 pode-se até identificar a frequência uma ou duas componentes, porém não é possível identificar claramente todas elas. Então como podemos fazer para identificá-las?

Você deve lembrar-se de que qualquer sinal periódico pode ser representado por uma função caracterizada por um somatório de funções senos e cossenos, denominada Série de Fourier. Ora, se podemos representar qualquer sinal por uma série de funções senos e cossenos, então podemos saber quais são as frequências de cada uma destas funções já que cada uma delas tem uma frequência bem definida. Isto pode ser feito realizando-se a *mudança de domínio* do dado, i.e., podemos trabalhar matematicamente o dado para ver características que não conseguimos ver no domínio em que o dado foi registrado.

Existem diversas ferramentas para isto dependendo do que se queira analisar no dado, porém aqui iremos trabalhar a ferramenta denominada **Transformada de Fourier**. Ela nos permitirá tirar o dado do *domínio do tempo* em que foi registrado e representá-lo no *domínio da frequência*. Em outras palavras, ela nos dá o poder de visualizar as componentes de frequência que compõem o sinal. Matematicamente ela é dada por

$$Y(f) = \int_{-\infty}^{\infty} y(t) e^{-2 \pi i f t} dt $$

em que a função $y(t)$ é o sinal registrado no domínio do tempo e $Y(f)$ nos dá o *espectro de amplitudes* do sinal, i.e., a amplitude de cada componente de frequência do presente no sinal original. É possível ainda partir de um dado no domínio da frequência e representá-lo no domínio do tempo fazendo a Transformada de Fourier inversa, dada por

$$y(t) = \int_{-\infty}^{\infty} Y(f) e^{2 \pi i f t} df $$

Esta ferramenta é extremamente importante e é a base para o processamento de sinais geofísicos. Ela pode ser aplicada tanto a dados adquiridos no domínio do tempo, como é o caso de um sinal sísmico ou de radar, quanto a dados adquiridos no domínio do espaço, como é o caso de dados adquiridos com os métodos potenciais.

Execute a célula abaixo para visualizar as cinco componentes de frequência do sinal sintético da Figura 1.

In [ ]:
frequencias, amplitude_f = mf.calcula_espectro(tempo, sinal)  # retorna o espectro completo (frequencias positivas e negativas)
mf.plota_espectro(frequencias, amplitude_f, n_figura='2')

**Pergunta:** Todas as frequências e amplitudes mostradas na Figura 2 correspondem às que foram estabelecidas no Passo 02?

# Passo 04 - Filtragem

Vamos agora modificar o sinal da Figura 2 removendo frequências específicas da sua composição e deixando as demais frequências inalteradas. Faremos isso através do processo de **Filtragem**. A filtragem pode ser executada no domínio da frequência através da multiplicação do espectro de amplitude por uma *função caixa* de amplitudes entre 0 e 1, assim aquelas frequências que forem multiplicadas pela amplitude 0 da função caixa serão eliminadas do espectro e as que forem multiplicadas pela amplitude 1 serão mantidas no espectro. Assim, ao realizar a transformada de Fourier inversa, o sinal será recomposto sem as componentes de frequência eliminadas.

Existem quatro tipos de filtros:
* **filtro passa-baixa**: em que é definida uma frequência de corte ($fc$) a partir da qual todas as componentes de frequência são eliminadas;
* **filtro passa-alta**: em que é definida uma frequência de corte ($fc$) a partir da qual todas as componentes de frequência são mantidas;
* **filtro passa-banda**: em que são definidas duas frequências de corte ($fc_1$ e $fc_2$) e todas as componentes de frequência entre elas são mantidas;
* **filtro rejeita-banda**: em que são definidas duas frequências de corte ($fc_1$ e $fc_2$) e todas as componentes de frequência entre elas são eliminadas;

Vamos verificar como estes filtros afetam nosso sinal sintético. Configure a função caixa de um filtro do tipo passa-banda em que serão mantidas as componentes do espectro entre as frequências de corte 5.0 Hz e 20.0Hz. Execute a célula abaixo para configurá-lo e visualizar o espectro do dado e a função caixa que acabou de ser configurada e como ficaria o espectro de amplitudes após sua multiplicação por esta função, i.e., após a filtragem.

**Obs.**: A amplitude da função caixa é apresentada aqui com amplitude maior do que 1. Isto é intencional e foi feito para que o topo da caixa fique acima de todas as componentes de frequência do espectro, para facilitar a compreensão da visualização. Nos cálculos a função caixa continua tendo amplitude máxima igual a 1.

In [ ]:
tipo = 'passa-banda'      # tipo de filtro a ser aplicado
f_corte = [5.0, 20.0]     # frequências de corte, em Hz (SEMPRE entre colchetes e separadas por vírgula se necessário)

#verifica_filtro(f_corte, tipo)
mf.plota_filtragem(amplitude_f, frequencias, f_corte, tipo, n_figura='3')

**Pergunta**: Houve alteração na amplitude ou na frequência de cada componente remanescente?

Por fim, vamos verificar como ficou o sinal filtrado, após removermos as demais componentes. Execute a célula abaixo para visualizar o dado após o cálculo da transformada de Fourier inversa do espectro da Figura 3b.

In [ ]:
sinal_filtrado = mf.executa_filtragem(amplitude_f, f_corte, tipo, frequencias)
mf.plota_gabarito(tempo, sinal_filtrado, sinal, n_figura='4')

# Passo 05 - Filtragem no "mundo real"

Até aqui trabalhamos com um sinal composto por apenas cinco componentes, porém um sinal ou dado geofísico real contém inúmeras componentes de frequência e selecioná-las não é tão simples. Vamos trabalhar agora com um dado real. O sinal que será utilizado é um traço que foi extraído de um perfil de GPR adquirido com antena cuja frequência nominal é de 200 MHz. Não é necessário que você conheça os detalhes teóricos do método agora. O importante neste momento é que você entenda a atuação da filtragem em um dado real, que funciona da mesma forma em qualquer tipo de dado geofísico.

Execute a célula abaixo para carregar este dado e visualizá-lo.

In [ ]:
dado_real = np.loadtxt('radar.txt')

tempo = dado_real[:, 0]
sinal_real = dado_real[:, 1]
mf.plota_sinal(tempo, sinal_real, n_figura='5')

A oscilação no início do traço é causada pelo acoplamento entre a antena e o solo. A oscilação entre 20 ns e 40 ns é causada por um objeto enterrado. A oscilação entre 50 ns e 60 ns é causada por um segundo objeto. As oscilações que aparecem após o tempo de 60 ns são decorrentes da perda de energia conforme o pulso eletromagnético se propaga em profundidade no material geológico, que foram levemente realçadas para mostrar a diferença no padrão de oscilações na segunda metade do traço. Você verá isto com calma na disciplina Geofísica II, portanto neste momento apenas tenha em mente que estamos interessados principalmente nas oscilações presentes ente 20 ns e 40 ns e entre 50 ns e 60 ns.

Vamos agora carregar este mesmo traço, porém contaminado por 50 componentes de frequências aleatórias, cada uma com uma amplitude também aleatória. Execute a célula abaixo para visualizar o dado contaminado por estas componentes.

In [ ]:
sinal_contaminado = dado_real[:, 2]
mf.plota_sinal(tempo, sinal_contaminado, n_figura='6')

**Pergunta**: Você ainda consegue identificar as oscilações de interesse com a mesma clareza de antes? Consegue perceber padrões de oscilação que se assemelhem a aqueles identificados como sendo de nosso interesse na Figura 5? 

Perceba como os padrões observados no dado sofreram mudanças drásticas em relação à Figura 5. É isto o que o *ruído* faz: ele mascara as informações que nos interessam e, portanto, deve ser removido para que possamos analisar o dado corretamente.

Execute a célula abaixo para visualizar o espectro de amplitudes do sinal contaminado pelo ruído.

In [ ]:
frequencias, amplitude_f = mf.calcula_espectro(tempo, sinal_contaminado)
mf.plota_espectro(frequencias, amplitude_f, n_figura='7')

Assim como fizemos anteriormente no Passo 04, vamos definir um filtro passa-banda para tentarmos eliminar as componentes de frequência que contaminaram o dado. Defina um filtro passa-banda entre as frequências de 200.0 X 10$^6$ Hz (i.e., 200.0 MHz) e 1.0 X 10$^9$ Hz (i.e., 1.0 GHz) na célula abaixo e, em seguida, execute-a para visualizar a localização do filtro sobre o espectro e o efeito da filtragem sobre as componentes de frequência:

In [ ]:
tipo = 'passa-banda'           # tipo de filtro a ser aplicado
f_corte = [200.0e6, 1.0e9]     # frequências de corte, em Hz (SEMPRE entre colchetes e separadas por vírgula se necessário)

#verifica_filtro(f_corte, tipo)
mf.plota_filtragem(amplitude_f, frequencias, f_corte, tipo, n_figura='8')

Por fim, vamos verificar como a remoção das frequências afetou o sinal no domínio do tempo. Execute a célula abaixo para mostrar uma comparação entre o sinal original (em azul claro) não contaminado pelas 50 componentes de frequência adicionais e o sinal após a filtragem configurada acima (em vermelho). Ficaram parecidos?

In [ ]:
sinal_filtrado = mf.executa_filtragem(amplitude_f, f_corte, tipo, frequencias)
mf.plota_gabarito(tempo, sinal_filtrado, sinal_real, n_figura='9')

Não, o dado ainda ficou bastante ruim. Isto significa que não removemos as componentes que estão contaminando o sinal. Como podemos saber então quais componentes devemos remover? Como sabemos quais componentes correspondem ao ruído e quais correspondem ao sinal que nos interessa? A resposta é: não sabemos com certeza. Temos apenas algumas dicas de em que região do espectro as componentes que correspondem ao sinal de interesse estão localizadas. Por exemplo, se sabemos que o dado foi adquirido com uma antena cuja frequência nominal é de 200 MHz, podemos começar eliminando as componentes que se distanciem desta região do espectro. Também podemos observar que há uma predominância de componentes de maior amplitude abaixo de 0.75 GHz, então é mais provável que as componentes do nosso sinal estejam abaixo desta frequência, mas ainda assim podemos ter componentes próximas desta região do espectro contribuindo construtivamente para o sinal que nos interessa. A filtragem é um processo de tentativa e erro: elimina-se uma determinada região do espectro e avalia-se a qualidade geral do dado, verificando se a "limpeza" executada permite enxergar o sinal de interesse.

Aqui foi estabelecido previamente que os intervalos entre 20 ns e 40 ns e entre 50 ns e 60 ns correspondem a oscilações de interesse, porém lembre-se de que em uma aquisição de dados geofísicos não se sabe exatamente o que existe em subsuperfície. Por este motivo, se o espectro for alterado de forma muito significativa, pode-se acabar excluindo o sinal de interesse juntamente com o ruído e isto leva a uma interpretação tão errada tanto quanto a interpretação feita a partir do dado não filtrado. Desta forma o processo de filtragem deve ser realizado de forma que se possa melhorar ao máximo a qualidade do dado, porém removendo-se o menor número possível de componentes de frequência para não degradar significativamente o sinal de interesse.